# Defining a Model in open_mcmc


## Simple Bayesian Model

Taking a Bayesian approach to modelling, we have prior beliefs about a parameter $h$, summarized by a prior distribution $f(h)$. Given $h$, observed data values are believed to be distributed according to $f(y|h)$ (the likelihood).

Using Bayes theorem, the posterior distribution for $h$ is then

$$ f(h | y ) \propto f(y | h) f(h)$$

In this example, we assume that both the prior and the likelihood are Normal distributions, with known precisions: i.e.
$$f(y | h) \sim N( h, \tau^{-1}) $$
and 
$$ f(h) \sim N( \mu, \lambda^{-1} )$$
where
* $\tau$  is the measurement precision for observations $y$
* $\mu$  is the prior mean for $h$
* $\lambda$  is the prior precision for $h$

## Setting up the model.

In the openmcmc package, a number of different types of distribution are available- in this example, we use only the Normal distribution.

The mean and precision parameters of the Normal distribution can be passed either as strings or as Parameter objects (these will be covered in later examples). Variables corresponding to the strings passed as distribution parameters must also be present in any state dictionary that is used for evaluation or estimation (see below).

In [1]:
# import modules required for the example
import numpy as np
from openmcmc.model import Model
from openmcmc.distribution.location_scale import Normal

The cell below defines a single `distribution.Normal` object.

In [2]:
my_dist = Normal('y', mean='h', precision='tau')

In the cell below, we define a `Model` object by passing multiple distributions as a list (corresponding to the likelihood and the prior).

In [3]:
mdl = Model([Normal('y', mean='h', precision='tau'),
             Normal('h', mean='mu', precision='lambda')])

The objects above define distributions or models, but in order to evaluate likelihoods or estimate parameters, we must pass a `state` dictionary which contains specific values for the parameters. A suitable `state` object for this example is defined below.

All items in the `state` dictionary are expected to have strings as keys, and values are expected to be `np.ndarray` objects which are at least 2D. The sizes used must be compatible for the desired operations.

In [4]:
state = {}
state['y']=np.array([150, 155, 190, 160, 173], ndmin=2)
state['h'] = np.array(180, ndmin=2)
state['tau'] = np.array(1 / 200, ndmin=2)
state['mu'] = np.array(160, ndmin=2)
state['lambda'] = np.array(1 / 100, ndmin=2)

state

{'y': array([[150, 155, 190, 160, 173]]),
 'h': array([[180]]),
 'tau': array([[0.005]]),
 'mu': array([[160]]),
 'lambda': array([[0.01]])}

## Making function calls

Having set up `my_dist` as the likelihood distribution above, we can generate random samples from it conditional on parameter values passed in `state`, as below:

In [5]:
my_dist.rvs(state, n=5)

array([[181.30730587, 179.18346712, 178.72074165, 171.80306536,
        183.87886684]])

Having set up `mdl` as a model containing both the likelihood and prior distributions, we can evaluate the log-posterior distribution (up to an additive constant) for the parameter values passed in `state`, as below:

In [6]:
mdl.log_p(state)

-28.24700970859217

In the same way, we can use the `grad_log_p` function (defined both for individual distributions and for models which combine distributions) to evaluate the gradient and the Hessian of the log-density/log-posterior at the `state` parameter values, as below:

In [7]:
gradient, hessian = mdl.grad_log_p(state, param='h')

print(gradient)
print(hessian)

[[-0.56]]
[[0.035]]
